<a href="https://colab.research.google.com/github/emisoft-designs/Data-Science-Practice-Projects/blob/main/FX_GBPJPY_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install talib-binary
import talib as ta
import pandas as pd
import numpy as np
import decimal as dec
from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 2.4 MB 4.9 MB/s 
Mounted at /content/drive


In [ ]:
file = "/content/drive/MyDrive/Colab Notebooks/Development lab/Forex Data/FX Historical Data/GBPJPY/GBPJPY15.csv"
dataset = pd.read_csv(file, names=['date','time','open','high','low','close','volume'])
print(dataset.head())

         date   time     open     high      low    close  volume
0  2019.08.19  16:00  129.121  129.220  129.111  129.208    1761
1  2019.08.19  16:15  129.208  129.219  129.156  129.157    1357
2  2019.08.19  16:30  129.156  129.174  129.051  129.140    1898
3  2019.08.19  16:45  129.140  129.176  129.073  129.094    1633
4  2019.08.19  17:00  129.096  129.142  129.040  129.124    2350


## **PREPARE DATA**

In [ ]:
close_float = [float(x) for x in dataset.close]
open_float = [float(x) for x in dataset.open]
high_float = [float(x) for x in dataset.high]
low_float = [float(x) for x in dataset.low]

close_price = np.array(close_float)
open_price = np.array(open_float)
high_price = np.array(high_float)
low_price = np.array(low_float)

In [ ]:
volume = dataset.volume/1000

In [ ]:
sma3    = pd.Series(ta.SMA(close_price, timeperiod=3))
sma10   = pd.Series(ta.SMA(close_price, timeperiod=10))
sma20   = pd.Series(ta.SMA(close_price, timeperiod=20))
sma50   = pd.Series(ta.SMA(close_price, timeperiod=50))
sma100  = pd.Series(ta.SMA(close_price, timeperiod=100))

SMA_10  = sma3 - sma10
SMA_20  = sma3 - sma20
SMA_50  = sma3 - sma50
SMA_100 = sma3 - sma100

In [ ]:
macd, signal, hist = ta.MACD(close_price, 12, 26, 9)
macd = macd - signal
macd

array([        nan,         nan,         nan, ..., -0.0875558 ,
       -0.06329529, -0.05149254])

In [ ]:
main, signal = pd.Series(ta.STOCH(high_price, low_price, close_price, fastk_period=14, slowk_period=2, slowk_matype=0, slowd_period=3, slowd_matype=0))

stochastic  = main - signal
stochastic

array([        nan,         nan,         nan, ...,  0.17506299,
       12.5744234 , 14.9292254 ])

In [ ]:
def getAction(o , c, r, sma):
  # determine buy action
  if o < sma and c > sma:
    return 'BUY'
  elif o > sma and c > sma:
    return 'HOLD_BUY'

  # determine sell action
  if o > sma and c < sma:
    return 'SELL'
  elif o < sma and c < sma:
    return 'HOLD_SELL'

  # no action detected
  return 'SEARCH'

In [ ]:
target_price = dataset.open.shift(5)
target_range = target_price - dataset.open

In [ ]:
action = [getAction(i, j, k, l) for i,j,k,l in zip(dataset.open, dataset.close, target_range, sma10)]
print(action)

['SEARCH', 'SEARCH', 'SEARCH', 'SEARCH', 'SEARCH', 'SEARCH', 'SEARCH', 'SEARCH', 'SEARCH', 'HOLD_BUY', 'HOLD_BUY', 'HOLD_BUY', 'HOLD_BUY', 'HOLD_BUY', 'SELL', 'BUY', 'BUY', 'HOLD_BUY', 'HOLD_BUY', 'HOLD_BUY', 'HOLD_BUY', 'HOLD_BUY', 'HOLD_BUY', 'HOLD_BUY', 'HOLD_BUY', 'HOLD_BUY', 'HOLD_BUY', 'SELL', 'BUY', 'SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'BUY', 'HOLD_BUY', 'HOLD_BUY', 'SELL', 'BUY', 'HOLD_BUY', 'HOLD_BUY', 'BUY', 'HOLD_BUY', 'HOLD_BUY', 'HOLD_BUY', 'BUY', 'HOLD_BUY', 'SELL', 'BUY', 'HOLD_BUY', 'SELL', 'HOLD_SELL', 'HOLD_SELL', 'BUY', 'SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'HOLD_SELL', 'BUY', 'SELL', 'HOLD_SELL', 'HOLD_

In [ ]:
dataset['volume']       = volume
dataset['SMA_10']       = SMA_10
dataset['SMA_20']       = SMA_20
dataset['SMA_50']       = SMA_50
dataset['SMA_100']      = SMA_100
dataset['MACD']         = macd
dataset['Stochastic']   = stochastic
dataset['action']       = action

In [ ]:
dataset = dataset.dropna()
dataset = dataset.drop(dataset.iloc[:,:2], axis=1)
dataset = dataset.reset_index(drop=True)

In [ ]:
dataset.head()

,open,high,low,close,volume,SMA_10,SMA_20,SMA_50,SMA_100,MACD,Stochastic,action
0,128.447,129.612,128.376,128.959,4.226,0.094833,0.120483,-0.197867,-0.412707,0.052252,-0.946630,BUY
1,128.961,129.180,128.820,129.082,5.970,0.205567,0.272017,-0.019373,-0.237113,0.076893,6.105069,HOLD_BUY
2,129.081,129.262,129.002,129.189,3.526,0.383067,0.487067,0.230967,0.010567,0.095910,9.359120,HOLD_BUY
3,129.191,129.205,128.855,128.898,3.114,0.327233,0.453383,0.219253,-0.007347,0.084788,-2.111341,HOLD_BUY
4,128.892,129.060,128.837,128.950,3.655,0.251533,0.392183,0.182113,-0.049907,0.077054,-8.968831,HOLD_BUY


# **DEEP LEARNING**

In [ ]:
action_dict = {'SEARCH':0, 'SELL':1, 'BUY':2, 'HOLD_SELL':3, 'HOLD_BUY':4}
dataset['action'] = dataset.action.apply(lambda x: action_dict[x])

In [ ]:
# get input and target dataset
X = dataset.iloc[:,:11]
Y = dataset.iloc[:,11:]

In [ ]:
# get sizes
tn_size = int(0.7*(X.index.size))
tt_size = int(0.25*(X.index.size))
vn_size = int(0.05*(X.index.size))

In [ ]:
# slice dataset to train, test and validation sets
train_X     = X.iloc[:tn_size,:]
train_Y     = Y.iloc[:tn_size,:]
test_X      = X.iloc[tn_size:(tt_size+tn_size),:]
test_Y      = Y.iloc[tn_size:(tt_size+tn_size),:]
validate_X  = X.iloc[(tt_size+tn_size):,:]
validate_Y  = Y.iloc[(tt_size+tn_size):,:]

In [ ]:

# restructure input format
train_x     = np.column_stack((train_X.open.values,train_X.high.values,train_X.low.values,train_X.close.values,train_X.volume.values,train_X.SMA_10.values,train_X.SMA_20.values,train_X.SMA_50.values,train_X.SMA_100.values,train_X.Stochastic.values,train_X.MACD.values))
test_x      = np.column_stack((test_X.open.values,test_X.high.values,test_X.low.values,test_X.close.values,test_X.volume.values,test_X.SMA_10.values,test_X.SMA_20.values,test_X.SMA_50.values,test_X.SMA_100.values,test_X.Stochastic.values,test_X.MACD.values))
validate_x  = np.column_stack((validate_X.open.values,validate_X.high.values,validate_X.low.values,validate_X.close.values,validate_X.volume.values,validate_X.SMA_10.values,validate_X.SMA_20.values,validate_X.SMA_50.values,validate_X.SMA_100.values,validate_X.Stochastic.values,validate_X.MACD.values))
train_y     = np.array(train_Y.values)
test_y      = np.array(test_Y.values)
validate_y  = np.array(validate_Y.values)

# **START TRAINING**

In [ ]:
import tensorflow as tf
from tensorflow import keras

model_loaded = True
score = 0.0
# load model
try:
  model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Development lab/Forex Data/model.h5')
  score = model.evaluate(test_x,test_y)[1]*100
  print("Model Loaded! Score: %.2f%%"%(score))
except OSError:
  model_loaded = False

if not model_loaded:
  # create and compile model if not created
  model = keras.models.Sequential([keras.layers.Dense(16, input_shape=(11,), activation='relu'), 
                                 keras.layers.Dense(10, activation='relu'),
                                 keras.layers.Dense(5,  activation='relu')])
  # compile
  model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])


In [ ]:
# Fit the model
if score < 80.1: model.fit(train_x, train_y, batch_size=15, shuffle=False, epochs=220)

Epoch 1/220
3029/3029 [==============================] - 5s 2ms/step - loss: 1.4442 - accuracy: 0.3956
Epoch 2/220
3029/3029 [==============================] - 5s 2ms/step - loss: 1.3227 - accuracy: 0.3794
Epoch 3/220
3029/3029 [==============================] - 5s 2ms/step - loss: 1.2819 - accuracy: 0.3606
Epoch 4/220
3029/3029 [==============================] - 5s 2ms/step - loss: 1.2199 - accuracy: 0.3879
Epoch 5/220
3029/3029 [==============================] - 5s 2ms/step - loss: 1.1942 - accuracy: 0.3898
Epoch 6/220
3029/3029 [==============================] - 5s 2ms/step - loss: 1.1834 - accuracy: 0.3872
Epoch 7/220
3029/3029 [==============================] - 5s 2ms/step - loss: 1.1783 - accuracy: 0.3860
Epoch 8/220
3029/3029 [==============================] - 5s 2ms/step - loss: 1.1440 - accuracy: 0.4123
Epoch 9/220
3029/3029 [==============================] - 5s 2ms/step - loss: 0.7833 - accuracy: 0.7005
Epoch 10/220
3029/3029 [==============================] - 5s 2ms/step - l

In [ ]:
# Evaluate
scores = model.evaluate(test_x,test_y)
score = scores[1]*100
print("%s: %.2f%%"% (model.metrics_names[1], score))

508/508 [==============================] - 1s 1ms/step - loss: 0.4673 - accuracy: 0.8147
accuracy: 81.47%


In [ ]:
# summarize model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                192       
                                                                 
 dense_1 (Dense)             (None, 10)                170       
                                                                 
 dense_2 (Dense)             (None, 5)                 55        
                                                                 
Total params: 417
Trainable params: 417
Non-trainable params: 0
_________________________________________________________________


# **MAKE PREDICTION**

In [ ]:
confidence = pd.DataFrame(model.predict(validate_x))
confidence.head()

,0,1,2,3,4
0,0.0,9.336584,8.253349,7.436961,8.346863
1,0.0,11.102986,8.822360,10.396685,6.803209
2,0.0,9.882450,9.938986,5.261714,12.971978
3,0.0,10.411502,14.677938,0.000000,19.739950
4,0.0,13.201462,14.496267,4.260456,19.992769


In [ ]:
prediction = [confidence.iloc[x].idxmax() for x in confidence.index]
ptable = np.column_stack((prediction,validate_y))

ptable = pd.DataFrame(ptable,columns=['Predicted','Expected'])
ptable.Predicted = ptable.Predicted.replace(np.nan,)
ptable['diff'] = ptable.Predicted - ptable.Expected 
print(ptable)

ptable = ptable.iloc[:,0].value_counts()
ptable = pd.DataFrame(ptable)
# diff = (ptable.iloc[1:].sum()/ptable.iloc[0:].sum())*100
# print('Difference = {}%'.format(diff))
ptable

      Predicted  Expected  diff
0             1         2    -1
1             1         4    -3
2             4         2     2
3             4         4     0
4             4         4     0
...         ...       ...   ...
3241          3         3     0
3242          3         3     0
3243          3         3     0
3244          3         3     0
3245          3         1     2

[3246 rows x 3 columns]


,Predicted
4,1549
3,1310
2,210
1,177


In [ ]:
diff = (ptable.iloc[0:]/ptable.iloc[0:].sum())*100
print('{}'.format(diff))

   Predicted
4  47.720271
3  40.357363
2   6.469501
1   5.452865


In [ ]:
x = np.column_stack(((161.575,161.659),(161.643,161.779),(161.542,161.631),(161.643,161.706),(0.618,1.034),(0.0300,0.0450),(0.019,0.0610),(0.0,0.0290),(0.3270,0.3560),(0.8362,14.911),(0.0078,0.014)))

In [ ]:
confidence = pd.DataFrame(model.predict(x))
prediction = [confidence.iloc[x].idxmax() for x in confidence.index]
confidence

,0,1,2,3,4
0,0.0,9.209517,10.159328,3.08198,13.420897
1,0.0,8.565049,12.919100,0.00000,17.907822


In [ ]:
# save model and architecture to single file
if score > 80.1:  
  model.save("/content/drive/MyDrive/Colab Notebooks/Development lab/Forex Data/GBPJPY15_model.h5")
  del model
  print("Saved model to disk")

Saved model to disk


In [ ]:
score

81.32010102272034